In [ ]:
contrib.columns

In [ ]:
contrib('pdf_url').duplicated().value_counts()

In [ ]:
contrib.loc[contrib.duplicated(Keep=False)].sort_values('pdf_url').head(10)['pdf_url']

In [ ]:
contrib = [('pdf_url')].duplicated().values_counts()

2. Do any of the tables need to become multiple tables, or combined into one table.

Should be combined into one table (share the same primary key): bioinfo, fec_ids, ideology.

In [2]:
members = pd.merge(bioinfo, fec_ids,
                   on ='bioguide_id',
                   how = 'outer',
                   validate = 'one_to_one',
                   inicator = 'matched')

NameError: name 'pd' is not defined

In [ ]:
members['matched'].value_counts()
#should be 545 

In [ ]:
members = members.drop('matched', axis=1)

In [ ]:
members = pd.mnerge(members, ideology,
                    on ='bioguide_id',
                   how = 'outer',
                   validate = 'one_to_one',
                   inicator = 'matched')

In [ ]:
members['matched'].value_counts()
#should be 545

In [ ]:
members = members.drop('matched', axis=1)

In [ ]:
members.columns

In [ ]:
members = members.drop(['Chamber', 'Party', 'District', 'State'], axis=1)
members.columns = [c.lower().replace(' ', '_') for c in members.columns]

In [ ]:
members.head(3).T

In [ ]:
members.columns = [c.lower().replace(' ', '_') for c in members.columns]

In [ ]:
members.head(3).T

In [ ]:
members.to_csv('../data/thirdNF/members.csv', index=False)

In [ ]:
for c in bill_summaries.columns:
    print(c)
    d = bill_summaries.groupby(['bill.type', 'bill.number']).agg({c: 'nunique'})
    print(np.mean(d[c]) == 1)

In [ ]:
bills = bill_summaries[['bill.type', 'bill.number','bill.congress', 'bill.originChamber', 
                       'bill.originChamberCode', 'bill.title', 'bill.updateDateIncludingText',
                       'bill.url']].drop_duplicates()
bills.columns = [c.lower().replace(' ', '_') for c in bills.columns]
bills.to.csv('../data/thirdNF/bills.csv')

In [ ]:
bill_versions = bill_summaries.drop(['bill.type', 'bill.number','bill.congress', 'bill.originChamber', 
                       'bill.originChamberode', 'bill.title', 'bill.updateDateIncludingText',
                       'bill.url'], axis=1)

In [ ]:
bill_versions .columns = [c.lower().replace(' ', '_') for c in bill_versions.columns]


In [3]:
vote_compare.head(3).T

NameError: name 'vote_compare' is not defined

In [ ]:
vote_compare.to_csv('../data/thirdNF/vote_compare.csv', index=False)